## ランダムフォレスト　デフォルト

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import tree


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
train_pkl.shape

(55369, 13)

In [4]:
train_pkl.describe()

,id,accommodates,bathrooms,bedrooms,host_response_rate,number_of_reviews,y,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,room_type_Private room,room_type_Shared room
count,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000
mean,27785.124745,3.156297,1.194766,1.265654,71.112283,13.975275,160.263830,0.257220,0.437302,0.001355,0.000235,0.413950,0.029114
std,16043.410949,2.154443,0.547776,0.850044,43.073836,29.867792,168.239385,0.437105,0.496058,0.036780,0.015321,0.492544,0.168127
min,0.000000,1.000000,0.000000,0.000000,0.000000,-128.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13889.000000,2.000000,1.000000,1.000000,0.000000,1.000000,74.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27778.000000,2.000000,1.000000,1.000000,100.000000,5.000000,111.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,41679.000000,4.000000,1.000000,1.000000,100.000000,20.000000,185.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
max,55582.000000,16.000000,8.000000,10.000000,100.000000,127.000000,1999.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 訓練データとテストデータに分割

In [5]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [6]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

## 訓練データで訓練

In [7]:
RF = RandomForestRegressor(random_state=42)

In [8]:
RF = RF.fit(train_X, train_y)

In [9]:
RF.feature_importances_

array([0.11785982, 0.17611408, 0.34347795, 0.08725802, 0.14984408,
       0.01697041, 0.02392551, 0.0006182 , 0.00513958, 0.06652161,
       0.01227075])

In [12]:
sorted(
    zip(map(lambda x: round(x, 3), RF.feature_importances_), train.iloc[:, [2, 3, 4, 5, 7, 8, 9, 10, 11]].columns),
    reverse=True)

[(0.343, 'host_response_rate'),
 (0.176, 'bedrooms'),
 (0.15, 'cancellation_policy_moderate'),
 (0.118, 'bathrooms'),
 (0.087, 'number_of_reviews'),
 (0.024, 'cancellation_policy_super_strict_30'),
 (0.017, 'cancellation_policy_strict'),
 (0.005, 'room_type_Private room'),
 (0.001, 'cancellation_policy_super_strict_60')]

In [13]:
# 訓練済みの決定木を視覚化
# dot_data = tree.export_graphviz(RF, out_file=None,
#                                feature_names=train.iloc[:, 1:].columns,
#                                class_names=train.Survived.name,
#                                rounded=True,
#                                filled=True,
#                                special_characters=True)

## テストデータで実行

In [14]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [15]:
test_x.shape, test_y.shape

((11074, 11), (11074,))

In [16]:
pred_y = RF.predict(test_x)

In [17]:
np.sqrt(mean_squared_error(test_y, pred_y))

132.15989892365315

## 検証データで実行

In [18]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [19]:
valid.shape

(18528, 12)

In [20]:
# ID の保存
valid_pass = valid.id.values

In [21]:
valid_X = valid.iloc[:, 1:].values

In [22]:
valid_X.describe()

,accommodates,bathrooms,bedrooms,host_response_rate,number_of_reviews,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,room_type_Private room,room_type_Shared room
count,18528.000000,18528.000000,18528.000000,18528.000000,18528.000000,18528.000000,18528.000000,18528.000000,18528.000000,18528.000000,18528.000000
mean,3.161863,1.189713,1.266947,70.916289,13.522722,0.257718,0.434801,0.001835,0.000216,0.411000,0.029253
std,2.155397,0.539284,0.860160,43.127724,29.909311,0.437390,0.495744,0.042799,0.014692,0.492028,0.168519
min,1.000000,0.000000,0.000000,0.000000,-128.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000,100.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,1.000000,1.000000,100.000000,20.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
max,16.000000,8.000000,10.000000,100.000000,127.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
valid_X.shape, train_X.shape

((18528, 11), (44295, 11))

In [24]:
pred_valid_y = RF.predict(valid_X)

In [25]:
pred_valid_y.shape

(18528,)

In [26]:
type(valid_pass), type(pred_valid_y)

(numpy.ndarray, numpy.ndarray)

In [27]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [28]:
result_df.head()

,y
0,250.081833
1,156.426354
2,116.605815
3,294.979645
4,155.867742


In [29]:
result_df.to_csv("./RF_3.csv", header=False)